In [1]:
import os
import pandas as pd
from tqdm import tqdm
from classes.vqa_model_predictor import DefaultVqaModelPredictor
from evaluate.VqaMedEvaluatorBase import VqaMedEvaluatorBase
from common.utils import VerboseTimer
import vqa_logger 
import logging
from pathlib import Path
import datetime
logger = logging.getLogger(__name__)

Using TensorFlow backend.


In [2]:
# %%capture
# mp = DefaultVqaModelPredictor.get_contender()

main_model = 172
# specialized_classifiers = {'Abnormality': 172, 'Modality': 157, 'Organ': 160, 'Plane': 159, 'Abnormality_yes_no':178}

#with class weights (looks worse):
# specialized_classifiers = {'Abnormality': 172, 'Modality': 157, 'Organ': 180, 'Plane': 179, 'Abnormality_yes_no':178} 
# notes = 'Notes: with class weights (looks worse)'

#with class weights (looks worse):
specialized_classifiers = {'Abnormality': 172, 'Modality': 184, 'Organ': 183, 'Plane': 159, 'Abnormality_yes_no':178} 
specialized_classifiers = {'Abnormality': 186, 'Modality': 184, 'Organ': 183, 'Plane': 188, 'Abnormality_yes_no':178} 

main_model = 185
specialized_classifiers = {'Abnormality': 185, 'Modality': 184, 'Organ': 183, 'Plane': 188, 'Abnormality_yes_no':178} 
notes= 'Words prediction for abnormality - Optimized for BLEU'
specialized_classifiers = {'Abnormality': 202, 'Modality': 184, 'Organ': 183, 'Plane': 188, 'Abnormality_yes_no':193} 
notes=''
specialized_classifiers = {'Abnormality': 202, 'Modality': 184, 'Organ': 70, 'Plane': 188, 'Abnormality_yes_no':193} 
notes=''

specialized_classifiers = {'Abnormality': 120, 'Modality': 184, 'Organ': 70, 'Plane': 188, 'Abnormality_yes_no':193} 
notes=''

mp = DefaultVqaModelPredictor(model=main_model, specialized_classifiers=specialized_classifiers)

[2019-05-04 22:24:19][classes.vqa_model_predictor][DEBUG] Getting model #185 (post_concat_dense_units: (8, 7, 6);
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: words
lstm_units: 128
batch_size: 32
epochs: 4
class weights: False
Inputs Attention: False


model_location = 'C:\Users\Public\Documents\Data\2019\models\20190430_0104_56_Abnormality_dense_8_7_6_lstm_128_trained\vqa_model.h5')
[2019-05-04 22:24:19][common.utils][DEBUG] Starting 'Loading Model'
[2019-05-04 22:24:23][common.utils][DEBUG] Loading Model: 0:00:04.813816
[2019-05-04 22:24:23][classes.vqa_model_predictor][WARNING] Expected main model to be with no question category, but got: "Abnormality"
[2019-05-04 22:24:23][classes.vqa_model_predictor][DEBUG] Getting model #120 (post_concat_dense_units: (8, 6);
Optimizer: RMSprop
loss: cosine_proximity
activation: sigmoid
prediction vector: answers
lstm_units: 0
batch_size: 32
Based on #68, with class weights

model_location = 'C:\Users\Public\Doc

In [3]:
mp

VqaModelPredictor(model=185, specialized_classifiers={'Abnormality': 120, 'Modality': 184, 'Organ': 70, 'Plane': 188, 'Abnormality_yes_no': 193})

In [4]:
dd = mp.df_validation
category = 'Abnormality'
add = dd[dd.question_category==category].answer.drop_duplicates()
len(add)
# dd.head()

411

In [5]:
mp.model_folder.prediction_data_name

'words'

In [6]:
# %%capture
datasets = {'test':mp.df_test, 'validation':mp.df_validation}
df_name_to_predict = 'test'


predictions = {}

for name, df in datasets.items():
    with VerboseTimer(f"Predictions for VQA contender {name}"):
        df_predictions = mp.predict(df)
        predictions[name] = df_predictions



predictions['validation'][:5]

[2019-05-04 22:24:36][common.utils][DEBUG] Starting 'Predictions for VQA contender test'
[2019-05-04 22:24:36][classes.vqa_model_predictor][INFO] For Category "Abnormality" using specialized classifier from:
ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190416_0322_14_dense_layers8_6_trained")
[2019-05-04 22:24:36][classes.vqa_model_predictor][DEBUG] Classifying: "Abnormality"
[2019-05-04 22:24:37][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-05-04 22:24:43][common.utils][DEBUG] Raw model prediction: 0:00:06.261760


Prediction: [509, 897, 904, 1332; probabilities: [0.012103817, 0.0004: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 229.53it/s]


[2019-05-04 22:24:44][classes.vqa_model_predictor][INFO] For Category "Modality" using specialized classifier from:
ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190429_0405_41_Modality_dense_8_6_trained")
[2019-05-04 22:24:44][classes.vqa_model_predictor][DEBUG] Classifying: "Modality"
[2019-05-04 22:24:44][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-05-04 22:24:48][common.utils][DEBUG] Raw model prediction: 0:00:03.600078


Prediction: [44]; probabilities: [0.9917121]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 125/125 [00:00<00:00, 221.44it/s]


[2019-05-04 22:24:48][classes.vqa_model_predictor][INFO] For Category "Organ" using specialized classifier from:
ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0440_18")
[2019-05-04 22:24:48][classes.vqa_model_predictor][DEBUG] Classifying: "Organ"
[2019-05-04 22:24:49][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-05-04 22:24:53][common.utils][DEBUG] Raw model prediction: 0:00:03.645403


...

Prediction: [14]; probabilities: [0.9491535]: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 209.15it/s]


[2019-05-04 22:25:39][classes.vqa_model_predictor][INFO] For Category "Abnormality_yes_no" using specialized classifier from:
ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190501_0350_23_Abnormality_yes_no_dense_9_6_trained")
[2019-05-04 22:25:39][classes.vqa_model_predictor][DEBUG] Classifying: "Abnormality_yes_no"
[2019-05-04 22:25:39][common.utils][DEBUG] Starting 'Raw model prediction'
[2019-05-04 22:25:41][common.utils][DEBUG] Raw model prediction: 0:00:01.877713


Prediction: [0]; probabilities: [0.9930084]: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 235.32it/s]


[2019-05-04 22:25:41][common.utils][DEBUG] Predictions for VQA contender validation: 0:00:43.503049


,image_name,question,answer,prediction,probabilities,path
index,,,,,,
12792,synpic54733.jpg,what imaging modality was used to take this im...,ct with iv contrast,ct with iv contrast,[(0.857)],C:\Users\Public\Documents\Data\2019\validation...
12793,synpic25647.jpg,what kind of scan is this?,xr - plain film,xr plain film,[(1.000)],C:\Users\Public\Documents\Data\2019\validation...
12794,synpic35681.jpg,was the mri taken with contrast?,no,no,[(0.657)],C:\Users\Public\Documents\Data\2019\validation...
12795,synpic39641.jpg,"is this a t1 weighted, t2 weighted, or flair i...",t2,t2,[(0.875)],C:\Users\Public\Documents\Data\2019\validation...
12796,synpic35693.jpg,is this a noncontrast mri?,yes,yes,[(0.562)],C:\Users\Public\Documents\Data\2019\validation...


In [7]:
outputs = {}
for name, df_predictions in predictions.items():
    curr_predictions = df_predictions.prediction.values
    df_predicted = datasets[name]
    df_output = df_predicted.copy()
    df_output['image_id'] = df_output.path.apply(lambda p: p.rsplit(os.sep)[-1].rsplit('.', 1)[0])
    df_output['prediction'] = curr_predictions

    columns_to_remove = ['path',  'answer_embedding', 'question_embedding', 'group', 'diagnosis', 'processed_answer']
    for col in columns_to_remove:
        del df_output[col]

    sort_columns = sorted(df_output.columns, key=lambda c: c not in ['question', 'prediction', 'answer'])
    df_output = df_output[sort_columns]    
    outputs[name] = df_output

In [8]:
df_output_test = outputs['test']
df_output_validation = outputs['validation']

In [9]:
display = df_output_validation[df_output_validation.question_category == 'Abnormality']
display = df_output_validation
display.sample(10)

,question,answer,prediction,image_name,processed_question,question_category,image_id
13745,what plane is this mammograph in?,mammo - mlo,3d reconstruction,synpic16484.jpg,what plane is this mammograph in,Plane,synpic16484
14657,what is most alarming about this mri?,posterior reversible encephalopathy syndrome (...,schwannoma,synpic44995.jpg,what is most alarming about this mri,Abnormality,synpic44995
13308,in what plane is this image oriented?,axial,axial,synpic38894.jpg,in what plane is this image oriented,Plane,synpic38894
14475,what is the primary abnormality in this image?,tethered spinal cord with terminal spinal lipoma,ependymoma,synpic19651.jpg,what is the primary abnormality in this image,Abnormality,synpic19651
13224,was iv contrast given to the patient?,yes,yes,synpic39853.jpg,was iv contrast given to the patient,Modality,synpic39853
14231,what organ system is being imaged?,vascular and lymphatic,genitourinary,synpic32126.jpg,what organ system is being imaged,Organ,synpic32126
14130,what organ system is pictured here?,skull and contents,skull and contents,synpic58993.jpg,what organ system is pictured here,Organ,synpic58993
13986,what organ system is evaluated primarily?,gastrointestinal,gastrointestinal,synpic34070.jpg,what organ system is evaluated primarily,Organ,synpic34070
14317,does this image look normal?,yes,yes,synpic28899.jpg,does this image look normal,Abnormality_yes_no,synpic28899
14086,which organ is captured by this mri?,vascular and lymphatic,musculoskeletal,synpic37599.jpg,which organ is captured by this mri,Organ,synpic37599


In [10]:
mp

VqaModelPredictor(model=185, specialized_classifiers={'Abnormality': 120, 'Modality': 184, 'Organ': 70, 'Plane': 188, 'Abnormality_yes_no': 193})

In [11]:
len(df_output_test), len(df_output_test.image_id.drop_duplicates())

(500, 500)

In [12]:
def get_str(df):
    strs = []
    debug_output_rows = df.apply(lambda row: row.image_id + '|'+ row.question + '|'+ row.prediction, axis=1 )
    output_rows = df.apply(lambda row: row.image_id + '|'+ row.prediction + '|'+row.answer, axis=1 )
    output_rows = output_rows.str.strip('|')
    rows = output_rows.values
    res = '\n'.join(rows)
    return res

res = get_str(df_output_test)
res_val = get_str(df_output_validation)


In [13]:
print(res[:200])

synpic54082|cta ct angiography
synpic48556|no
synpic50696|iv
synpic37194|us doppler ultrasound
synpic31308|an angiogram
synpic24739|t2
synpic32365|yes
synpic47936|iv
synpic29792|yes
synpic43561|no
syn


### Get evaluations per category:

In [14]:
evaluations = {}
pbar = tqdm(df_output_validation.groupby('question_category'))
for question_category, df in pbar:        
    pbar.set_description(f'evaluating {len(df)} for {question_category} items')
    curr_predictions = df.prediction.values
    curr_ground_truth = df.answer.values
    curr_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=curr_predictions, ground_truth=curr_ground_truth)
    evaluations[question_category] = curr_evaluations    

   


evaluating 500 for Plane items: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:05<00:00,  1.69s/it]


### Get Total Evaluation:

In [15]:
total_evaluations = VqaMedEvaluatorBase.get_all_evaluation(predictions=df_output_validation.prediction.values, ground_truth=df_output_validation.answer.values)    
evaluations['Total'] = total_evaluations

In [16]:
evaluations
df_evaluations = pd.DataFrame(evaluations).T#.sort_values(by=('bleu'))
df_evaluations['sort'] = df_evaluations.index == 'Total'
df_evaluations = df_evaluations.sort_values(by = ['sort', 'wbss'])
del df_evaluations['sort']
df_evaluations

,bleu,strict_accuracy,wbss
Abnormality,0.021749,0.018868,0.043482
Organ,0.708409,0.700000,0.699596
Modality,0.750807,0.712000,0.702513
Plane,0.740000,0.740000,0.743488
Abnormality_yes_no,0.826087,0.826087,0.834267
Total,0.564491,0.552000,0.556364


In [17]:
model_repr = repr(mp)
model_repr
sub_models = {category: folder for category, (model, folder) in mp.model_by_question_category.items()}
sub_models_str = '\n'.join([str(f'{category}: {folder} ({folder.prediction_data_name})') for category, folder in sub_models.items() if folder is not None])

model_description_copy = df_evaluations.copy()

def get_prediction_vector(category):
    sub_model = sub_models.get(category)
    if sub_model is not None:
        return sub_model.prediction_data_name
    else:
        return '--'
    
model_description_copy['prediction_vector'] = model_description_copy.index.map(get_prediction_vector)


model_description =f'''
==Model==
{model_repr}

==Submodels==
{sub_models_str}

==validation evaluation==
{model_description_copy.to_string()}

==Notes==
{notes}
'''


print(model_description)




==Model==
VqaModelPredictor(model=185, specialized_classifiers={'Abnormality': 120, 'Modality': 184, 'Organ': 70, 'Plane': 188, 'Abnormality_yes_no': 193})

==Submodels==
Abnormality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190416_0322_14_dense_layers8_6_trained") (answers)
Modality: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190429_0405_41_Modality_dense_8_6_trained") (answers)
Organ: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190329_0440_18") (answers)
Plane: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190430_2147_46_Plane_dense_8_trained") (answers)
Abnormality_yes_no: ModelFolder(folder="C:\\Users\\Public\\Documents\\Data\\2019\\models\\20190501_0350_23_Abnormality_yes_no_dense_9_6_trained") (answers)

==validation evaluation==
                        bleu  strict_accuracy      wbss prediction_vector
Abnormality         0.021749         0.018868  0.043482          

In [18]:
import time
now = time.time()
ts = datetime.datetime.fromtimestamp(now).strftime('%Y%m%d_%H%M_%S')
submission_base_folder = Path('C:\\Users\\Public\\Documents\\Data\\2019\\submissions')
submission_folder = submission_base_folder/ts
submission_folder.mkdir()


txt_path = submission_folder/f'submission_{ts}.txt'
txt_path.write_text(res)

txt_path_val = submission_folder/f'submission_{ts}_validation.txt'
txt_path_val.write_text(res_val)


model_description_path = submission_folder/f'model_description.txt'
model_description_path.write_text(model_description)

with pd.HDFStore(str(submission_folder/ 'predictions.hdf')) as store:
    for name, df_predictions in predictions.items():
        store[name] = df_predictions

In [19]:
print(name)
df_predictions.sample(5)

validation


,image_name,question,answer,prediction,probabilities,path
index,,,,,,
14417,synpic43843.jpg,what is most alarming about this angiogram?,arteriovenous malformation (avm),schwannoma,[(0.116)],C:\Users\Public\Documents\Data\2019\validation...
13357,synpic49985.jpg,what plane is this ct scan in?,sagittal,axial,[(0.799)],C:\Users\Public\Documents\Data\2019\validation...
14110,synpic57642.jpg,what organ system is visualized?,skull and contents,skull and contents,[(0.393)],C:\Users\Public\Documents\Data\2019\validation...
14299,synpic48040.jpg,is there an abnormality in the ct scan?,no,no,[(0.655)],C:\Users\Public\Documents\Data\2019\validation...
14482,synpic47049.jpg,what is most alarming about this ct scan?,hemangiomas of the liver,schwannoma,[(0.116)],C:\Users\Public\Documents\Data\2019\validation...


In [20]:
idx = 14525
series = df_predictions.loc[idx]
series.answer, series.prediction ,series.probabilities 

('pericardial cyst', 'meningioma', array(['(0.116)'], dtype=object))

In [21]:
mp

VqaModelPredictor(model=185, specialized_classifiers={'Abnormality': 120, 'Modality': 184, 'Organ': 70, 'Plane': 188, 'Abnormality_yes_no': 193})